# Music Popularity Analysis

<hr style="border:2px solid black"> </hr>

## Notebook 02 - Spotify API for Album Search & Track Retrieval

---

### Import libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import sys

---
### Import Barcodes for Spotify URI Retrieval

In [2]:
mb_db_isrcs = np.array(pd.read_csv('mb_db_isrcs.csv', names=['isrc'], header=None))

In [3]:
CLIENT_ID = '75855396ac0446279df54c8a7df71414'
CLIENT_SECRET = '7bc71c24b81743ebbb79f0b612c26587'

In [4]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [5]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [6]:
## Test of requests method
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
r.json()

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [50]:
isrc_uri = []
j = 0

In [7]:
# Initiate from file
isrc_uri = pd.read_csv('isrc_uri.csv')[['isrc', 'sp_uri']]
isrc_uri = isrc_uri.values.tolist()
isrc_uri = [(a[0], a[1]) for a in isrc_uri]
j = len(isrc_uri)

In [8]:
# actual GET request with proper header
for j in range(j, len(mb_db_isrcs)):
    isrc = mb_db_isrcs[j]
    
    retries = 1
    success = False
    while not success:
        try:
            r = requests.get(BASE_URL + 'search?query=isrc%3A'+isrc[0]+'&type=track&locale=en-US%2Cen%3Bq%3D0.9&offset=0&limit=20', headers=headers)
            success = True
        except Exception as e:
            wait = retries * 30;
            print(f'Error! Waiting {wait} secs and re-trying...')
            sys.stdout.flush()
            time.sleep(wait)
            retries += 1
    
    resp_header = r.headers
    if len(r.json()['tracks']['items']) > 0:
        isrc_uri.append((isrc[0], r.json()['tracks']['items'][0]['id']))
    time.sleep(np.random.random()*1.5)
    j += 1
    if j % 100 == 0:
        print("{} isrcs searched, {} songs found".format(j, len(isrc_uri)))
        
    if j % 1000 == 0:
        # POST
        auth_response = requests.post(AUTH_URL, {
            'grant_type': 'client_credentials',
            'client_id': CLIENT_ID,
            'client_secret': CLIENT_SECRET,
        })

        # convert the response to JSON
        auth_response_data = auth_response.json()

        # save the access token
        access_token = auth_response_data['access_token']
        
        headers = {
            'Authorization': 'Bearer {token}'.format(token=access_token)
        }

44100 isrcs searched, 44096 songs found
44200 isrcs searched, 44184 songs found
44300 isrcs searched, 44273 songs found
44400 isrcs searched, 44364 songs found
44500 isrcs searched, 44448 songs found
44600 isrcs searched, 44534 songs found
44700 isrcs searched, 44617 songs found
44800 isrcs searched, 44699 songs found
44900 isrcs searched, 44785 songs found
45000 isrcs searched, 44872 songs found
45100 isrcs searched, 44965 songs found
45200 isrcs searched, 45053 songs found
45300 isrcs searched, 45139 songs found
45400 isrcs searched, 45227 songs found
45500 isrcs searched, 45320 songs found
45600 isrcs searched, 45404 songs found
45700 isrcs searched, 45497 songs found
45800 isrcs searched, 45587 songs found
45900 isrcs searched, 45674 songs found
Error! Waiting 30 secs and re-trying...
46000 isrcs searched, 45767 songs found
46100 isrcs searched, 45857 songs found
46200 isrcs searched, 45946 songs found
46300 isrcs searched, 46033 songs found
46400 isrcs searched, 46127 songs found


KeyboardInterrupt: 

In [31]:
r.headers

{'cache-control': 'private, max-age=0', 'retry-after': '52261', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Wed, 20 Apr 2022 13:23:30 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}

In [9]:
pd.DataFrame(isrc_uri, columns=['isrc', 'sp_uri']).to_csv('isrc_uri.csv')